Current tasks
-------------
- Set _eval_energy and eval_energy to be nogil
- ~~Make CompiledModel a cdef class~~
- Link CompiledModel-based energy evaluation into the main solver
- Decide how/if to store gradients and Hessians of RedlichKisterSums
- Write analagous eval_gradient and eval_hessian functions
- Link gradient and Hessian for CompiledModel into main solver

In [1]:
from pycalphad.core.compiled_model import CompiledModel
from pycalphad import calculate, Database, Model
from pycalphad.core.phase_rec import PhaseRecord
import pycalphad.variables as v
import numpy as np

In [2]:
dbf = Database('alcocr-sandbox.tdb')

In [3]:
cmpmdl = CompiledModel(dbf, ['AL', 'CO', 'CR', 'VA'], 'BCC_A2')
out = np.zeros(7)
cmpmdl.eval_energy_gradient(out, np.array([101325, 600, 0.3, 0.3, 0.4-1e-12, 1e-12, 1]), np.array([]))
out

array([  0.00000000e+00,  -9.05365885e+00,  -1.01755913e+03,
        -1.01755913e+03,   4.17600426e+02,  -1.32854175e+05,
        -6.76268123e+03])

In [7]:
mdl = Model(dbf, ['AL', 'CO', 'CR', 'VA'], 'BCC_A2')
[mdl.GM.diff(x).subs({v.T: 600., v.P:101325., v.Y('BCC_A2', 0, 'AL'): 0.3, v.Y('BCC_A2', 0, 'CO'): 0.3,
                    v.Y('BCC_A2', 0, 'CR'): 0.4-1e-12, v.Y('BCC_A2', 0, 'VA'): 1e-12, v.Y('BCC_A2', 1, 'VA'): 1.})
 for x in [v.P, v.T, v.Y('BCC_A2', 0, 'AL'),v.Y('BCC_A2', 0, 'CO'),v.Y('BCC_A2', 0, 'CR'),v.Y('BCC_A2', 0, 'VA'), v.Y('BCC_A2', 1, 'VA')]]

[0,
 -9.05365884524735,
 -1017.55912894182,
 -1017.55912894182,
 417.600425887349,
 -138286.370348319,
 -16296.5859214615]

In [5]:
%time res = calculate(dbf, ['AL', 'CO', 'CR', 'VA'], sorted(dbf.phases.keys()), T=2000, P=101325, output='GM', pdens=200, model=CompiledModel)
res.GM[..., :20]

CPU times: user 3.01 s, sys: 20 ms, total: 3.03 s
Wall time: 3.03 s


<xarray.DataArray 'GM' (P: 1, T: 1, points: 20)>
array([[[ -1.24620355e+05,  -1.40654061e+05,  -1.23950630e+05,
          -1.41731195e+05,  -1.24840486e+05,  -1.43867091e+05,
          -1.33280011e+05,  -1.30571011e+05,  -1.35062664e+05,
          -1.32621793e+05,  -1.28914793e+05,  -1.27018480e+05,
          -1.42832019e-06,  -1.42832019e-06,  -4.76098411e+05,
          -1.42832019e-06,  -5.25675953e+02,  -9.35087210e+02,
          -1.30035020e+03,  -1.63679237e+03]]])
Coordinates:
  * T        (T) float64 2e+03
  * P        (P) float64 1.013e+05
  * points   (points) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19

In [6]:
%time res2 = calculate(dbf, ['AL', 'CO', 'CR', 'VA'], sorted(dbf.phases.keys()), T=2000, P=101325, output='GM', pdens=200, model=Model)
res2.GM[..., :20]

CPU times: user 4.21 s, sys: 159 ms, total: 4.37 s
Wall time: 22.4 s


<xarray.DataArray 'GM' (P: 1, T: 1, points: 20)>
array([[[ -1.24620355e+05,  -1.40654061e+05,  -1.23950630e+05,
          -1.41731195e+05,  -1.24840486e+05,  -1.43867091e+05,
          -1.33280011e+05,  -1.30571011e+05,  -1.35062664e+05,
          -1.32621793e+05,  -1.28914793e+05,  -1.27018480e+05,
          -1.42832019e-06,  -1.42832019e-06,  -1.42831649e-06,
          -1.42832019e-06,  -5.25675953e+02,  -9.35087210e+02,
          -1.30035020e+03,  -1.63679237e+03]]])
Coordinates:
  * T        (T) float64 2e+03
  * P        (P) float64 1.013e+05
  * points   (points) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19